**Important thing**: for running everything correctly, you should run one cell per time; hence do not click on Kernel and Restart and clear output, but use the button **Run** for all cell; once you have done that, you can simply set a new goal, everytime, by using the boxes on the third last jupyter cell.

 Here I included all the libraries regard ros.

In [ ]:
import rospy
from geometry_msgs.msg import Pose, Point
import following_goal.msg
import actionlib
import actionlib.msg
from following_goal.srv import NumberGoals
from following_goal.msg import PlanningAction
from std_srvs.srv import *
from nav_msgs.msg import Odometry



This import is useful for using widget library.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

I initialized my ros node, and connected it to the server.

In [ ]:
global user, status

rospy.init_node('user_interface')
#define the the action client
user=actionlib.SimpleActionClient('/reaching_goal', following_goal.msg.PlanningAction)
#waiting for connection to the server
user.wait_for_server()
#get the state
status=user.get_state()

Here I defined the widgets for the user interface and also for the data which have collected by the robot. 

In [ ]:

#boxes for setting the goal
posx_widget =widgets.FloatText( description='Pos x:', disabled=False,)
posy_widget =widgets.FloatText( description='Pos y:', disabled=False,)

#button for confirming goals
confirm_goal=widgets.Button(description="Confirm goal", button_style="success")
output_confirm=widgets.Output()

#button for deleting goals
cancel_target=widgets.Button(description="Cancel a goal", button_style="danger")
output_cancel= widgets.Output()

#boxes for displaying current position and distance from the closest obstacle
min_dist_widget =widgets.FloatText( description='Minimum distance from obstacle:', disabled=True, style= {'description_width': 'initial'})
current_px_widget=widgets.FloatText(description='Current posx:', disabled=True)
current_py_widget=widgets.FloatText(description='Current posy:', disabled=True)

menu_1=widgets.VBox([posx_widget, posy_widget])
menu_2=widgets.VBox([confirm_goal, cancel_target])

box_menu_1=widgets.Output(layout={'border': '1px solid black'})
box_menu_2=widgets.Output(layout={'border': '3px solid green'})

    
    


These libraries are useful for plotting the robot trajectory and the goal, that is set before by the user.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import rospy
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation

This class has the task for plotting and updating anything concerning the trajectory , current position and number of reached and deleted goals of robot

In [ ]:
import time
posx=0 
posy=0 
global user, status
goals=['reached', 'deleted']
colors=['green', 'red']

class Visualiser:
    def __init__(self):
        global posx, posy
        self.fig, (self.ax, self.ax2) = plt.subplots(1,2, figsize=(10,4))
        self.ln, = self.ax.plot([], [], color='black', markersize='2', label='Trajectory')
        #here I set the legend on the right of the graph
        self.ax.plot(posx, posy, marker='o', color='green', markersize='5', label='Target reached')
        self.ax.plot(posx, posy, marker='o', color='gray', markersize='5', label='Current target')
        self.ax.plot(posx, posy, marker='o', color='red', markersize='5', label='Deleted target')
        self.posx=posx
        self.posy=posy
        self.x_data, self.y_data = [] , []
        #variables for counting number of reaching and deleting goals
        self.reached = 0
        self.deleted = 0
        #boolean variables for displaying the correct value on the graph
        self.plot_reached=False
        self.plot_deleted=False
    def plot_init(self):
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        self.ax2.set_ylim(0, 15)
        return self.ln
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        #take all time the current position
        current_px_widget.value=round(msg.pose.pose.position.x, 2)
        current_py_widget.value=round(msg.pose.pose.position.y, 2)
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.posx=posx
        self.posy=posy
        status=user.get_state()
        #check if status is reached
        if status==3 and not(self.plot_reached):
            #change the colour of the goal
            self.ax.plot(self.posx, self.posy, marker='o', color='green', markersize='5', label='Target')
            self.reached+=1
            #display the bar chart
            self.ax2.bar(goals, [self.reached, self.deleted], color = colors)
            self.plot_reached=True
        #check if status is deleted
        elif status==2 and not(self.plot_deleted):
            #change the colour of the goal
            self.ax.plot(self.posx, self.posy, marker='o', color='red', markersize='5', label='Target')
            self.deleted+=1
            #display the bar chart
            self.ax2.bar(goals, [self.reached, self.deleted], color = colors)
            self.plot_deleted=True
        elif (not(status==3) and self.plot_reached):
            self.plot_reached=False 
        elif (not(status==2) and self.plot_deleted):
            self.plot_deleted=False
        elif (not(status==3) and not(status==2)):
            self.ax.plot(self.posx, self.posy, marker='o', color='gray', markersize='5', label='Target')
        
        self.ax2.bar(goals, [self.reached, self.deleted], color =colors)
        return self.ln

This is the user interface, whereby the user can set a new goal, in the specific boxes, and confirm or delete it.

In [ ]:
global user, status

#function for catching the click of the button confirm_goal
def on_confirm_clicked(confirm_goal):
    global posx, posy
    posx=float(posx_widget.value)
    posy=float(posy_widget.value)
    with output_confirm:
        send_goal()
confirm_goal.on_click(on_confirm_clicked)

#function for sending the goal to the server
def send_goal():
    goal = following_goal.msg.PlanningGoal()
    status=user.get_state()
    if not (status==actionlib.GoalStatus.ACTIVE):
        goal.target_pose.pose.position.x = posx
        goal.target_pose.pose.position.y = posy
        user.send_goal(goal)
        print("Goal sent!")
    else:
        print("Goal has been already set")

#function for catching the click of the button cancel_target
def on_cancel_clicked(cancel_target):
    with output_cancel:
        cancel_function()
cancel_target.on_click(on_cancel_clicked)

#function for deleting the goal
def cancel_function():
    status=user.get_state()
    if status==actionlib.GoalStatus.ACTIVE:
        user.cancel_goal()
        print("Goal cancelled")
    else:
        print("No goal has been set")

with box_menu_1:
    print("Here you can set a new goal:")
    display(menu_1)
    print("Click one of the two buttons for confirming or deleting the goal:")
    display(menu_2, output_confirm, output_cancel)

box_menu_1

Here I show the plot of robot trajectory, bar chart of reached and deleted goals and finally some boxes, which show the current position and distance from the closest obastacle.

In [ ]:


vis = Visualiser()
vis.ax.set_title('Plot of robot trajectory')
vis.ax.set_xlabel('Position on x')
vis.ax.set_ylabel('Position on y')
vis.ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
vis.ax.grid()

vis.ax2.set_title('Number of goal reached and deleted')
vis.ax2.set_xlabel('Goals')
vis.ax2.set_ylabel('Number of goal')

sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

with box_menu_2:
    display(min_dist_widget)
    display(current_px_widget)
    display(current_py_widget)
box_menu_2


Thie code below is a simple callback, which compute each time the distance from the closest obstacle,thanks to the subscription from the topic /scan.

In [ ]:
import time
import rospy
import math
import jupyros as jr
from sensor_msgs.msg import LaserScan

global min_range

#callback for finding the distance from the closest obstacle
def scan_callback(scan):
    min_range = scan.range_max
    for range_val in scan.ranges:
        if range_val < min_range:
            min_range = range_val
    min_dist_widget.value=round(min_range, 2)

#this widget lets you to start and stop the subscritpion, if you want
jr.subscribe('/scan', LaserScan, scan_callback)
